<a href="https://colab.research.google.com/github/Chitrak07/Kaggle/blob/main/santa2022_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shubhamkumarvaish","key":"e9b84c4d3ac13462bad15f7f3412e375"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c santa-2022

100% 809k/809k [00:00<00:00, 1.01MB/s]
100% 809k/809k [00:00<00:00, 1.01MB/s]


In [7]:
!unzip /content/santa-2022.zip

Archive:  /content/santa-2022.zip
  inflating: image.csv               
  inflating: image.png               
  inflating: sample_submission.csv   


In [21]:
import numpy as np, pandas as pd

# Read image as a numpy array:
df_image = pd.read_csv('/content/image.csv')
side = df_image.x.nunique()
radius = df_image.x.max()
image = df_image[['r','g','b']].values.reshape(side,side,-1)

# Flip X axis and transpose X-Y axes to simplify cartesian to array mapping:
image = image[::-1,:,:]
image = np.transpose(image, (1, 0, 2))

In [23]:
from functools import reduce

def get_position(config):
    return reduce(lambda p, q: (p[0] + q[0], p[1] + q[1]), config, (0, 0))

def compress_path(path):
    
    if len(path) > 2:
        
        new_path = []
        
        max_conf_dist = 1
        
        r = [[] for _ in range(len(path[0]))]
        
        for p in path:
            for i, c in enumerate(p):

                if len(r[i]) == 0 or r[i][-1] != c:
                    
                    if c not in r[i]:
                        r[i].append(c)
                    else:
                        r[i] = r[i][:r[i].index(c) + 1]
                        
                    assert r[i][-1] == c
        
        max_conf_dist = max([len(r_) for r_ in r])
        
        for i in range(max_conf_dist):
            
            new_conf = []
            
            for _, r_ in enumerate(r):
                
                if i < len(r_):
                    c_ = r_[i]
                else:
                    c_ = r_[-1]
                
                new_conf.append(c_)
            
            new_path.append(new_conf)
            
        return new_path
                               
        
    return path

def rotate_link(vector, direction):
    x, y = vector
    if direction == 1:  # counter-clockwise
        if y >= x and y > -x:
            x -= 1
        elif y > x and y <= -x:
            y -= 1
        elif y <= x and y < -x:
            x += 1
        else:
            y += 1
    elif direction == -1:  # clockwise
        if y > x and y >= -x:
            x += 1
        elif y >= x and y < -x:
            y += 1
        elif y < x and y <= -x:
            x -= 1
        else:
            y -= 1
    return (x, y)

def rotate(config, i, direction):
    config = config.copy()
    config[i] = rotate_link(config[i], direction)
    return config

def get_direction(u, v):
    """Returns the sign of the angle from u to v."""
    direction = np.sign(np.cross(u, v))
    if direction == 0 and np.dot(u, v) < 0:
        direction = 1
    return direction

def color_cost(from_position, to_position, image, color_scale=3.0):
    return np.abs(image[to_position] - image[from_position]).sum() * color_scale

def get_path_to_point(config, point):
    """Find a path of configurations to `point` starting at `config`."""
    path = [config]
    # Rotate each link, starting with the largest, until the point can
    # be reached by the remaining links. The last link must reach the
    # point itself.
    for i in range(len(config)):
        link = config[i]
        base = get_position(config[:i])
        relbase = (point[0] - base[0], point[1] - base[1])
        position = get_position(config[:i+1])
        relpos = (point[0] - position[0], point[1] - position[1])
        radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
        # Special case when next-to-last link lands on point.
        if radius == 1 and relpos == (0, 0):
            config = rotate(config, i, 1)
            if get_position(config) == point:  # Thanks @pgeiger
                path.append(config)
                break
            else:
                continue
        while np.max(np.abs(relpos)) > radius:
            direction = get_direction(link, relbase)
            config = rotate(config, i, direction)
            path.append(config)
            link = config[i]
            base = get_position(config[:i])
            relbase = (point[0] - base[0], point[1] - base[1])
            position = get_position(config[:i+1])
            relpos = (point[0] - position[0], point[1] - position[1])
            radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
    assert get_position(path[-1]) == point
    
    path = compress_path(path)
    
    return path

def get_path_to_configuration(from_config, to_config):
    path = [from_config]
    config = from_config.copy()
    while config != to_config:
        for i in range(len(config)):
            config = rotate(config, i, get_direction(config[i], to_config[i]))
        path.append(config)
    assert path[-1] == to_config
    
    path = compress_path(path)
    
    return path

def config_to_string(config):
    return ';'.join([' '.join(map(str, vector)) for vector in config])

In [24]:
import numpy as np, pandas as pd

# Read image as a numpy array:
df_image = pd.read_csv('/content/image.csv')
side = df_image.x.nunique()
radius = df_image.x.max()
image = df_image[['r','g','b']].values.reshape(side,side,-1)

# Flip X axis and transpose X-Y axes to simplify cartesian to array mapping:
image = image[::-1,:,:]
image = np.transpose(image, (1, 0, 2))

In [29]:
# Prepare pixel travel map:
unvisited = np.ones([side,side]) # one = unvisited pixel; 0 = visited pixel
total = side*side - 1 #total number of pixels minus the origin
origin = [(64,0),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(-1,0)] #origin configuration
config = origin.copy() #future configuration

# Output header and origin configuration:
f = open('submission1.csv', 'w')
print('configuration', file=f) #header
print(config_to_string(origin), file=f) #origin configuration

# Output arrows for visualization:
a = open('arrows1.csv', 'w')
print('x,y,dx,dy', file=a) #header

# Continue until all locations have been visited:
while(total):
    
    # Optimization variables:
    cost = 1e6
    distance = 1e6
    found = False
    
    # Current configuration:
    base = get_position(config)
    base_arr = (base[0]+radius, base[1]+radius)
    unvisited[base_arr] = 0
    
    # Is the location one step below unvisited?
    if base[1]==-128: #if we reached the bottom border
        below = 0
    else:
        below = unvisited[(base_arr[0],base_arr[1]-1)]
    
    # Single-link step:
    for i in range(len(origin)): #for each arm link
        for d in [-1,1]: #for each direction
            # Rotate link and get new position and vertical displacement:
            config2 = rotate(config, i, d)
            pos = get_position(config2)
            dy = pos[1] - base[1]
            
            # Convert from cartesian to array coordinates and measure cost:
            pos_arr = (pos[0]+radius, pos[1]+radius)
            cost2 = 1 + color_cost(base_arr, pos_arr, image)
            
            # Must move down unless impossible:
            if unvisited[pos_arr] and cost2<cost and (dy<0 or (dy>=0 and below==0)): 
                config_next = config2.copy()
                cost = cost2
                found = True

    # Double-link step:
    for i in range(len(origin)-1):
        for d1 in [-1,1]:
            for j in range(i+1,len(origin)):
                for d2 in [-1,1]:
                    # Rotate two separate links, get position and vertical displacement:
                    config2 = rotate(config, i, d1)
                    config2 = rotate(config2, j, d2)
                    pos = get_position(config2)
                    dy = pos[1] - base[1]
                    
                    # Convert from cartesian to array coordinates and measure cost:
                    pos_arr = (pos[0]+radius, pos[1]+radius)
                    cost2 = np.sqrt(2) + color_cost(base_arr, pos_arr, image)
                    
                    # Must move down unless impossible:
                    if(unvisited[pos_arr] and cost2 < cost and below==0): 
                        config_next = config2.copy()
                        cost = cost2
                        found = True
                        
    # If an unvisited point was found, we are done for this step:
    if found:
        config = config_next.copy()
        pos = get_position(config)
        total -= 1
        
        # Print configuration and arrows:
        print(config_to_string(config), file=f)
        print(base[0],',',base[1],',',pos[0]-base[0],',',pos[1]-base[1], file=a)
        
    # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
    else:
        # Search every single pixel of the travel map for unvisited points:
        for i in range(side): 
            for j in range(side): 
                if unvisited[(i,j)]:
                    
                    # Measure the distance to the current point and choose the nearest one:
                    #changed base_arr[1] to 0
                    distance2 = np.sqrt((base_arr[0]-i)**2 + (base_arr[1]-j)**2)
                    if(distance2 < distance):
                        point = (i-radius, j-radius)
                        distance = distance2
                        
        # Go to the nearest unvisited point:
        path = get_path_to_point(config, point)[1:]
        
        # Output shortest trajectory:
        for config in path:
            pos = get_position(config)
            pos_arr = (pos[0]+radius, pos[1]+radius)
            
            # Update the travel map:
            if unvisited[pos_arr]:
                unvisited[pos_arr] = 0
                total -= 1
                
            # Print configuration and arrows:
            print(config_to_string(config), file=f)
            print(base[0],',',base[1],',',pos[0]-base[0],',',pos[1]-base[1], file=a)
            base = pos

# Return to origin:
base = get_position(config)
path = get_path_to_configuration(config, origin)[1:]

# Output return trajectory:
for config in path:
    pos = get_position(config)
    
    # Print configuration and arrows:
    print(config_to_string(config), file=f)
    print(base[0],',',base[1],',',pos[0]-base[0],',',pos[1]-base[1], file=a)
    base = pos

# Close output files:
f.close()
a.close()

KeyboardInterrupt: ignored

In [26]:
gnuplot

NameError: ignored

In [27]:
set terminal jpeg size 5000,5000
set output 'plot.png'
set key autotitle columnhead
set datafile separator ','
set size ratio -1
plot '/content/image.csv' u 1:2:($3*256):($4*256):($5*256) w rgbimage,\
'arrows.csv' u 1:2:3:4 w vectors

SyntaxError: ignored